In [1]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rn
import time 
from sklearn.decomposition import RandomizedPCA
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import chi2_kernel
from scipy import linalg
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, pairwise
from sklearn.cluster import KMeans
from scipy.io import loadmat
from collections import Counter
from scipy.interpolate import Rbf

In [2]:
import json
with open('feature_data.json', 'r') as f:
    features = json.load(f)

features_ar = np.array(features)

In [3]:
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
camIds = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()
filelists = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()

train_idxss = np.subtract(train_idxs, 1)
gallery_idxss = np.subtract(gallery_idxs, 1)
query_idxss = np.subtract(query_idxs, 1)

In [4]:
camIds_train = camIds[train_idxss]
filelists_train = filelists[train_idxss]
labels_train = labels[train_idxss]
features_train = features_ar[train_idxss]

camIds_gal = camIds[gallery_idxss]
filelists_gal = filelists[gallery_idxss]
labels_gal = labels[gallery_idxss]
features_gal = features_ar[gallery_idxss]

camIds_query = camIds[query_idxss]
filelists_query = filelists[query_idxss]
labels_query = labels[query_idxss]
features_query = features_ar[query_idxss]

In [5]:
def scores_query(ranklist, query_l, instances, k):
    acc = 0
    TP = 0
    FP = 0
    prec = 0
    rec = 0

    if rankKeval(ranklist, query_l) == True:
        acc = 1
        
    for i in range (0, k):
        if (ranklist[i] == query_l):
            TP = TP + 1
        else:
            FP = FP +1
    prec = TP/(TP + FP)
    rec = TP/instances
    
    return(acc, prec, rec)

In [6]:
def rankKeval(labels_neigh, label_query):
    contained = False
    for i in range (0, len(labels_neigh)):
        if (labels_neigh[i] == label_query):
            contained = True
    return(contained)

In [10]:
def kNNpolykernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, degree, gamma, k):
    acc = 0
    n_query = len(labels_query)
    n_gal = len(labels_gal)
    rbf_gq = pairwise.polynomial_kernel(features_query, features_gal, degree=degree, gamma=gamma)
    rbf_gg = pairwise.polynomial_kernel(features_gal, features_gal, degree=degree, gamma=gamma)
    rbf_qq = pairwise.polynomial_kernel(features_query, features_query, degree=degree, gamma=gamma)
    rbfm = np.empty([n_query, n_gal])
    for m in range (0, n_query):
        for z in range (0, n_gal):
            rbfm[m][z] = rbf_gg[z][z] + rbf_qq[m][m] - 2*rbf_gq[m][z]
    for i in range(0, n_query):
        dist = np.copy(rbfm[[i], :])[0]
        ids = np.copy(camIds_gal)
        lbs = np.copy(labels_gal)
        inds = list(range(0, n_gal))
        neigh = []
        while len(neigh)<k:
            rem = False
            mind = np.argmin(dist)
            ind = inds[mind]
            if ((ids[mind] == camIds_query[i]) and (lbs[mind] == labels_query[i])):
                rem = True
            if rem == False:
                neigh = np.append(neigh, [lbs[mind]], axis=0)
            dist = np.delete(dist, mind, axis=0)
            inds = np.delete(inds, mind, axis=0)
            ids = np.delete(ids, mind, axis=0)
            lbs = np.delete(lbs, mind, axis=0)

        accq, precq, recq = scores_query(neigh, labels_query[i], 1, k) # CHANGE INSTANCES HERE
        acc = acc + accq
    acc = acc/n_query
    print('acc = ', acc)
    
    return(acc)

In [11]:
for i in np.logspace(-3, 3, num=10, endpoint=True, base=10.0):
    for m in list(range(1, 6)):
        print('i = ', i)
        print('m = ', m)
        acc = kNNpolykernel(features_gal, labels_gal, camIds_gal, features_query, labels_query, m, i, 1)

i =  0.001
m =  1
acc =  0.47
i =  0.001
m =  2
acc =  0.46714285714285714
i =  0.001
m =  3
acc =  0.4585714285714286
i =  0.001
m =  4
acc =  0.45285714285714285
i =  0.001
m =  5
acc =  0.4421428571428571
i =  0.004641588833612777
m =  1
acc =  0.47
i =  0.004641588833612777
m =  2
acc =  0.46
i =  0.004641588833612777
m =  3
acc =  0.4421428571428571
i =  0.004641588833612777
m =  4
acc =  0.4057142857142857
i =  0.004641588833612777
m =  5
acc =  0.3657142857142857
i =  0.021544346900318832
m =  1
acc =  0.47
i =  0.021544346900318832
m =  2
acc =  0.45571428571428574
i =  0.021544346900318832
m =  3
acc =  0.4235714285714286
i =  0.021544346900318832
m =  4
acc =  0.3692857142857143
i =  0.021544346900318832
m =  5
acc =  0.30857142857142855
i =  0.1
m =  1
acc =  0.47
i =  0.1
m =  2
acc =  0.455
i =  0.1
m =  3
acc =  0.42214285714285715
i =  0.1
m =  4
acc =  0.3585714285714286
i =  0.1
m =  5
acc =  0.29
i =  0.46415888336127775
m =  1
acc =  0.47
i =  0.46415888336127775
m =